In [1]:
import pandas as pd
import numpy as np

# Set options to show dfs in full
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

## Clean LTC data

In [2]:
# Convert name and additional info columns to lowercase
df_genLTC = pd.read_csv('df_final_ngan.csv')
df_genLTC['cleaned_name'] = df_genLTC['name'].str.lower()
df_genLTC['additional_info'] = df_genLTC['additional_info'].str.lower()

# Adjust names from general LTC database to match names from LTC COVID database
df_genLTC["cleaned_name"].replace({"vision '74 inc.": "vision nursing home", "finlandia hoivakoti nursing home limited": "finlandia hoivakoti nursing home", "pinecrest nursing home - bobcaygeon": "pinecrest nursing home (bobcaygeon)", "residence saint- louis": "residence saint-louis", "st patrick's home": "st. patrick's home"}, inplace=True)
df_genLTC.head()

,name,address,city and postal code,LHIN,licensee,management,home type,beds,short stay,residents council,family council,accreditation,additional_info,city,postal code,number of beds,total_inspections,quartiles_total_range,quartiles_total_rank,5y_inspections,quartiles_5y_range,quartiles_5y_rank,2y_inspections,quartiles_2y_range,quartiles_2y_rank,cleaned_name
0,AFTON PARK PLACE LONG TERM CARE COMMUNITY,1200 Afton Drive,"Sarnia, N7S6L6",Erie St. Clair,S & R Nursing Homes Ltd.,NaN,For-Profit,Home with approximately 128 beds,No,Yes,Yes,Yes,NaN,Sarnia,N7S6L6,128.0,67.0,"(46.0, 172.0]",3.0,34.0,"(23.0, 86.0]",3.0,9.0,"(7.0, 11.0]",2.0,afton park place long term care community
1,"ALBRIGHT GARDENS HOMES, INCORPORATED",5050 Hillside Drive,"Beamsville, L0R1B2",Hamilton Niagara Haldimand Brant (Hnhb),"Albright Gardens Homes, Incorporated",NaN,Non-Profit,Home with approximately 231 beds,No,Yes,Yes,No,NaN,Beamsville,L0R1B2,231.0,39.0,"(31.0, 46.0]",2.0,25.0,"(23.0, 86.0]",3.0,15.0,"(11.0, 44.0]",3.0,"albright gardens homes, incorporated"
2,ALEXANDER PLACE,329 Parkside Drive P. O. Box 50,"Waterdown, L0R2H0",Hamilton Niagara Haldimand Brant (Hnhb),Waterdown Long Term Care Centre Inc.,NaN,For-Profit,Home with approximately 128 beds,Yes,Yes,Yes,Yes,NaN,Waterdown,L0R2H0,128.0,28.0,"(21.0, 31.0]",1.0,17.0,"(16.0, 23.0]",2.0,8.0,"(7.0, 11.0]",2.0,alexander place
3,ALGOMA DISTRICT HOMES FOR THE AGED (ALGOMA MANOR),135 Dawson Street,"Thessalon, P0R1L0",North East,Board Of Management For The District Of Algoma,NaN,NaN,Home with approximately 108 beds,Yes,Yes,Yes,No,home closed on 11/01/2011,Thessalon,P0R1L0,108.0,5.0,"(0.0, 21.0]",0.0,NaN,NaN,NaN,NaN,NaN,NaN,algoma district homes for the aged (algoma manor)
4,ALGOMA MANOR NURSING HOME,145 Dawson Street,"Thessalon, P0R1L0",North East,Algoma Manor Nursing Home,NaN,NaN,Home with approximately 96 beds,Yes,Yes,Yes,Yes,NaN,Thessalon,P0R1L0,96.0,23.0,"(21.0, 31.0]",1.0,14.0,"(11.0, 16.0]",1.0,7.0,"(5.0, 7.0]",1.0,algoma manor nursing home


## Review records of homes that have been closed

In [3]:
# We see that 20 homes are closed, and one home was merged
df_genLTC['additional_info'].value_counts()
df_genLTC.additional_info.str.contains("closed", na=False).value_counts()

False    631
True     20 
Name: additional_info, dtype: int64

In [4]:
# Drop rows with homes that are closed
df_genLTC = df_genLTC[~df_genLTC.additional_info.str.contains("closed", na=False)].reset_index()

# review other additional info, there are 5 homes with additional info
df_genLTC.loc[df_genLTC['additional_info'].notnull()]


,index,name,address,city and postal code,LHIN,licensee,management,home type,beds,short stay,residents council,family council,accreditation,additional_info,city,postal code,number of beds,total_inspections,quartiles_total_range,quartiles_total_rank,5y_inspections,quartiles_5y_range,quartiles_5y_rank,2y_inspections,quartiles_2y_range,quartiles_2y_rank,cleaned_name
25,26,BENDALE ACRES,2920 Lawrence Avenue East,"Scarborough, M1P2T8",Central East,Toronto Long-term Care Homes And Services,NaN,Municipal,Home with approximately 302 beds,Yes,Yes,Yes,Yes,"pavillon omer deslauriers, 37 beds, long term care services provided in french and english.",Scarborough,M1P2T8,302.0,35.0,"(31.0, 46.0]",2.0,17.0,"(16.0, 23.0]",2.0,10.0,"(7.0, 11.0]",2.0,bendale acres
114,118,CRAIGHOLME,221 Main Street R. R. #1,"Ailsa Craig, N0M1A0",South West,Craigwiel Gardens,NaN,Non-Profit,Home with approximately 83 beds,Yes,Yes,Yes,No,stutti@craigwielgardens.on.ca,Ailsa Craig,N0M1A0,83.0,51.0,"(46.0, 172.0]",3.0,21.0,"(16.0, 23.0]",2.0,10.0,"(7.0, 11.0]",2.0,craigholme
121,126,DEARNESS HOME FOR SENIOR CITIZENS,710 Southdale Road East,"London, N6E1R8",South West,The Corporation Of The City Of London,NaN,Municipal,Home with approximately 243 beds,Yes,Yes,Yes,Yes,"datars bere, dearness home managing director",London,N6E1R8,243.0,76.0,"(46.0, 172.0]",3.0,33.0,"(23.0, 86.0]",3.0,16.0,"(11.0, 44.0]",3.0,dearness home for senior citizens
453,469,SARSFIELD COLONIAL HOME,2861 Colonial Road P.O. Box 130,"Sarsfield, K0A3E0",Champlain,2629693 Ontario Inc.,Taminagi Inc. (As Manager),For-Profit,Home with approximately 46 beds,No,Yes,Yes,Yes,recipient no 346524 – sarsfield colonial home\r\ncomment - change in ownership effective february 19 2020. details transferred to recipient no 664183/ facility no nh1692\r\n,Sarsfield,K0A3E0,46.0,62.0,"(46.0, 172.0]",3.0,42.0,"(23.0, 86.0]",3.0,12.0,"(11.0, 44.0]",3.0,sarsfield colonial home
487,504,"ST. JOSEPH'S HEALTH CARE, LONDON - MOUNT HOPE CENTRE FOR LONG TERM CARE - MARIAN VILLA",200 College Avenue P.O. Box 5777,"London, N6A1Y1",South West,"St. Joseph's Health Care, London",NaN,Non-Profit,Home with approximately 217 beds,Yes,Yes,Yes,Yes,"home was merged with mount hope long-term care centre effective 1st january, 2016. please see mount hope long-term care centre for information.",London,N6A1Y1,217.0,68.0,"(46.0, 172.0]",3.0,17.0,"(16.0, 23.0]",2.0,NaN,NaN,NaN,"st. joseph's health care, london - mount hope centre for long term care - marian villa"


## Manual Review of one home that was merged in Jan 2016

- Address to the merged home is just a general PO BOX to the licensee
- Will remove the merged home from the database

In [5]:
# show rows with information on the 2 homes that were merged
df_genLTC[df_genLTC['cleaned_name'].str.contains('mount hope', regex=False, na=False)]

,index,name,address,city and postal code,LHIN,licensee,management,home type,beds,short stay,residents council,family council,accreditation,additional_info,city,postal code,number of beds,total_inspections,quartiles_total_range,quartiles_total_rank,5y_inspections,quartiles_5y_range,quartiles_5y_rank,2y_inspections,quartiles_2y_range,quartiles_2y_rank,cleaned_name
360,373,MOUNT HOPE CENTRE FOR LONG TERM CARE,21 Grosvenor Street P.O. Box 5777,"London, N6A1Y6",South West,"St. Joseph's Health Care, London",NaN,Non-Profit,Home with approximately 177 beds,Yes,Yes,Yes,Yes,NaN,London,N6A1Y6,177.0,87.0,"(46.0, 172.0]",3.0,44.0,"(23.0, 86.0]",3.0,15.0,"(11.0, 44.0]",3.0,mount hope centre for long term care
487,504,"ST. JOSEPH'S HEALTH CARE, LONDON - MOUNT HOPE CENTRE FOR LONG TERM CARE - MARIAN VILLA",200 College Avenue P.O. Box 5777,"London, N6A1Y1",South West,"St. Joseph's Health Care, London",NaN,Non-Profit,Home with approximately 217 beds,Yes,Yes,Yes,Yes,"home was merged with mount hope long-term care centre effective 1st january, 2016. please see mount hope long-term care centre for information.",London,N6A1Y1,217.0,68.0,"(46.0, 172.0]",3.0,17.0,"(16.0, 23.0]",2.0,NaN,NaN,NaN,"st. joseph's health care, london - mount hope centre for long term care - marian villa"


In [6]:
df_genLTC.iloc[487]

index                    504                                                                                                                                            
name                     ST. JOSEPH'S HEALTH CARE, LONDON - MOUNT HOPE CENTRE FOR LONG TERM CARE - MARIAN VILLA                                                         
address                  200 College Avenue P.O. Box 5777                                                                                                               
city and postal code     London, N6A1Y1                                                                                                                                 
LHIN                     South West                                                                                                                                     
licensee                 St. Joseph's Health Care, London                                                                                                  

In [8]:
# drop row with the merged home
df_genLTC = df_genLTC.drop(df_genLTC.index[487])

# check to see it is gone
df_genLTC[df_genLTC['cleaned_name'].str.contains('mount hope', regex=False, na=False)]

,index,name,address,city and postal code,LHIN,licensee,management,home type,beds,short stay,residents council,family council,accreditation,additional_info,city,postal code,number of beds,total_inspections,quartiles_total_range,quartiles_total_rank,5y_inspections,quartiles_5y_range,quartiles_5y_rank,2y_inspections,quartiles_2y_range,quartiles_2y_rank,cleaned_name
360,373,MOUNT HOPE CENTRE FOR LONG TERM CARE,21 Grosvenor Street P.O. Box 5777,"London, N6A1Y6",South West,"St. Joseph's Health Care, London",NaN,Non-Profit,Home with approximately 177 beds,Yes,Yes,Yes,Yes,NaN,London,N6A1Y6,177.0,87.0,"(46.0, 172.0]",3.0,44.0,"(23.0, 86.0]",3.0,15.0,"(11.0, 44.0]",3.0,mount hope centre for long term care


## Merge general LTC and covid LTC data

In [9]:
df_covidLTC = pd.read_csv('merged_ltc.csv')
df_covidLTC['cleaned_name'] = df_covidLTC['LTC Home'].str.lower()
df_covidLTC.head()
df_covidLTC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 9 columns):
Unnamed: 0                  244 non-null int64
LTC Home                    244 non-null object
City                        244 non-null object
Beds                        244 non-null int64
Confirmed Resident Cases    180 non-null object
Resident Deaths             244 non-null object
Confirmed Staff Cases       180 non-null object
Status                      244 non-null object
cleaned_name                244 non-null object
dtypes: int64(2), object(7)
memory usage: 17.2+ KB


In [10]:
# check that homes from COVID database are present in the general LTC homes database
a = set(df_genLTC['cleaned_name'])
b = set(df_covidLTC['cleaned_name'])
def returnNotMatches(a, b):
    return [[x for x in b if x not in a]]
returnNotMatches(a, b)

[[]]

In [11]:
# merge the LTC database with the LTC COVID database
df_merged = pd.merge(left=df_genLTC, right=df_covidLTC, how='outer', left_on='cleaned_name', right_on='cleaned_name')

In [12]:
df_merged.to_csv(r'merge_genltc_covidltc_FINAL.csv', index = False)

## Import and prep ODHF data

In [13]:
# Import Open Database of Healthcare Facilities csv
odhf = pd.read_csv('odhf_v1.csv', engine='python')

# Filter for facilities in ON
df_on = odhf[odhf['province'].isin(['on'])].reset_index()

# Clean facilities name
import unicodedata
df_on['cleaned_name'] = df_on['facility_name'].str.lower().apply(lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode())
df_on.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3733 entries, 0 to 3732
Data columns (total 19 columns):
level_0                      3733 non-null int64
index                        3733 non-null int64
facility_name                3733 non-null object
source_facility_type         3705 non-null object
odhf_facility_type           3728 non-null object
provider                     3733 non-null object
unit                         0 non-null object
street_no                    3122 non-null object
street_name                  3154 non-null object
postal_code                  3726 non-null object
city                         3724 non-null object
province                     3733 non-null object
source_format_str_address    271 non-null object
CSDname                      3540 non-null object
CSDuid                       3151 non-null float64
Pruid                        3733 non-null int64
latitude                     3159 non-null float64
longitude                    3159 non-null float

In [ ]:
# df_on[df_on['cleaned_name'].str.contains('health care london', regex=False, na=False)]

In [ ]:
# home = df_on.iloc[[3049]]
# home['cleaned_name']

## List of homes in the LTC database that are not listed in the ODHF

In [14]:
a = set(df_on['cleaned_name'])
b = set(df_merged['cleaned_name'])
def returnNotMatches(a, b):
    return [x for x in b if x not in a]
missing_odhf = returnNotMatches(a, b)

In [15]:
len(missing_odhf)


24

In [16]:
missing_odhf.sort()
missing_odhf

['albright gardens homes, incorporated',
 'bella senior care residences inc.',
 'bon air long term care residence',
 'caressant care cobden',
 'caressant care harriston',
 'champlain long term care residence',
 'heartwood (fka versa-care cornwall)',
 'lady isabelle nursing home',
 'lancaster long term care residence',
 'malden park continuing care centre',
 'niagara long term care residence',
 'north renfrew long-term care services',
 'north shore health network – eldcap unit',
 'north shore health network – ltc unit',
 'people care centre',
 'residence saint-louis',
 'santé manitouwadge health',
 "st. joseph's health centre, guelph",
 "st. joseph's mother house (martha wing)",
 "st. joseph's villa, dundas",
 'strathcona long term care',
 'the meadows',
 'william a. "bill" george extended care facility',
 'élisabeth-bruyère residence']

## Replace cleaned names in the LTC and ODHF df

In [17]:
df_merged_names = df_merged['cleaned_name'].replace({
    'albright gardens homes, incorporated' : 'albright gardens',
    'bella senior care residences inc.':'bella senior care residences',
    'bon air long term care residence':'chartwell bon air long term care residence',
    'caressant care cobden':'caressant care - cobden',
    'caressant care harriston':'caressant care - harriston',
    'champlain long term care residence':'chartwell champlain long term care residence',
    'heartwood (fka versa-care cornwall)':'heartwood',
# lady isabelle MISSING, SEE NOTE 
    'lancaster long term care residence':'chartwell lancaster long term care residence',
# malden MISSING, SEE NOTE 
    'niagara long term care residence':'chartwell niagara long term care residence',
    'north renfrew long-term care services':'north renfrew long-term care services inc.',
    'north shore health network – eldcap unit':'north shore health network - eldcap unit' ,
    'north shore health network – ltc unit' : 'north shore health network - ltc unit',
    'people care centre':'peoplecare  stratford',
# residence saint-louis, OK
    'santé manitouwadge health':'sante manitouwadge health',
    'st. joseph\'s health centre, guelph':'st. joseph\'s health centre - guelph',
    'st. joseph\'s mother house (martha wing)':'st. josephs motherhouse martha wing',
    'st. joseph\'s villa, dundas' : "st. joseph's villa (dundas)",
# strathcona long term care, MISSING SEE NOTE
    'the meadows':'revera inc.  the meadows long term care centre',
    'william a. "bill" george extended care facility': 'william a. \'bill\' george extended care facility',
    'élisabeth-bruyère residence':'elisabeth-bruyere residence'
})

df_on_names = df_on['cleaned_name'].replace({'lisabeth-bruyre residence': 'elisabeth-bruyere residence',
                                             'rsidence saint-louis' : 'residence saint-louis'})

In [18]:
a = set(df_on_names)
b = set(df_merged_names)
def returnNotMatches(a, b):
    return [x for x in b if x not in a]
missing_odhf_names = returnNotMatches(a, b)
print(len(missing_odhf_names))
missing_odhf_names.sort()
missing_odhf_names

3


['lady isabelle nursing home',
 'malden park continuing care centre',
 'strathcona long term care']

## Manual review of discrepancies between LTC data and ODHF
### Missing from ODHF
1. lady isabelle nursing home: http://www.ladyisabelle.ca/Contact_Information.html, MISSING from ODHF
2. MALDEN PARK CONTINUING CARE CENTRE: seems to be closed, last inspection 2010, telephone disconnected
3. Strathcona Long Term Care, https://svch.ca/long-term-care/strathcona/location/, has replaced saugeen valley nursing center in ODHF which is at the same address, https://svch.ca/saugeen-valley-nursing-centre-announces-plan-to-redevelop-87-bed-facility-in-mount-forest/#.Xr2rvRNKjfY

### Other notes:
- people care centre: listed in ODHF but no geodata, https://www.peoplecare.ca/
- sante manitouwadge health: called and this is part of the hospital with 9 LTC beds

### Therefore we will remove Malden LTC from LTC database, change Saugeen Valley to Strathcona in ODHF before merging the LTC data with ODHF

In [19]:
# Show information/index on Malden LTC home
df_merged[df_merged['cleaned_name'].str.contains('malden', regex=False, na=False)]
df_merged.iloc[321]

# Drop row with malden LTC from merged LTC database
df_merged = df_merged.drop(df_merged.index[321])

# Replace saugeen with strathcona in ODHF
df_on['cleaned_name'].replace({'saugeen valley nursing center': 'strathcona long term care'})

# df_on[df_on['cleaned_name'].str.contains('saugeen', regex=False, na=False)]
# df_on.iloc[2721]



0       (georgian, trillium & sunset manors) corporation of the county of simcoe                   
1       147 elder street inc.                                                                      
2       2 spirited people of the 1st nations                                                       
3       3 c's reintroduction centre, the                                                           
4       rsidence st. franois                                                                       
5       a better way retirement home corp.                                                         
6       a-way express courier service                                                              
7       aamjiwnaag first nation chippewas of sarnia (css)                                          
8       aamjiwnaang first nation chippewas of sarnia                                               
9       abbeylawn manor retirement home                                                            


## Merge LTC data with ODHF

In [20]:
df_all = pd.merge(left=df_merged, right=df_on, 
                  how='left', left_on='cleaned_name', 
                  right_on='cleaned_name')
df_all.info()
df_all.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 629 entries, 0 to 628
Data columns (total 53 columns):
index_x                      629 non-null int64
name                         629 non-null object
address                      629 non-null object
city and postal code         629 non-null object
LHIN                         627 non-null object
licensee                     627 non-null object
management                   72 non-null object
home type                    626 non-null object
beds                         627 non-null object
short stay                   627 non-null object
residents council            627 non-null object
family council               627 non-null object
accreditation                627 non-null object
additional_info              4 non-null object
city_x                       629 non-null object
postal code                  629 non-null object
number of beds               627 non-null float64
total_inspections            628 non-null float64
quartiles_total

,index_x,name,address,city and postal code,LHIN,licensee,management,home type,beds,short stay,residents council,family council,accreditation,additional_info,city_x,postal code,number of beds,total_inspections,quartiles_total_range,quartiles_total_rank,5y_inspections,quartiles_5y_range,quartiles_5y_rank,2y_inspections,quartiles_2y_range,quartiles_2y_rank,cleaned_name,Unnamed: 0,LTC Home,City,Beds,Confirmed Resident Cases,Resident Deaths,Confirmed Staff Cases,Status,level_0,index_y,facility_name,source_facility_type,odhf_facility_type,provider,unit,street_no,street_name,postal_code,city_y,province,source_format_str_address,CSDname,CSDuid,Pruid,latitude,longitude
0,0,AFTON PARK PLACE LONG TERM CARE COMMUNITY,1200 Afton Drive,"Sarnia, N7S6L6",Erie St. Clair,S & R Nursing Homes Ltd.,NaN,For-Profit,Home with approximately 128 beds,No,Yes,Yes,Yes,NaN,Sarnia,N7S6L6,128.0,67.0,"(46.0, 172.0]",3.0,34.0,"(23.0, 86.0]",3.0,9.0,"(7.0, 11.0]",2.0,afton park place long term care community,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2434.0,2435.0,afton park place long term care community,long-term care home,Nursing and residential care facilities,Province of Ontario,NaN,1200,afton drive,N7S6L6,sarnia,on,NaN,Sarnia,3538030.0,35.0,42.976325,-82.361444
1,1,"ALBRIGHT GARDENS HOMES, INCORPORATED",5050 Hillside Drive,"Beamsville, L0R1B2",Hamilton Niagara Haldimand Brant (Hnhb),"Albright Gardens Homes, Incorporated",NaN,Non-Profit,Home with approximately 231 beds,No,Yes,Yes,No,NaN,Beamsville,L0R1B2,231.0,39.0,"(31.0, 46.0]",2.0,25.0,"(23.0, 86.0]",3.0,15.0,"(11.0, 44.0]",3.0,"albright gardens homes, incorporated",0.0,"Albright Gardens Homes, Incorporated",Beamsville,231.0,NaN,0,NaN,Inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ALEXANDER PLACE,329 Parkside Drive P. O. Box 50,"Waterdown, L0R2H0",Hamilton Niagara Haldimand Brant (Hnhb),Waterdown Long Term Care Centre Inc.,NaN,For-Profit,Home with approximately 128 beds,Yes,Yes,Yes,Yes,NaN,Waterdown,L0R2H0,128.0,28.0,"(21.0, 31.0]",1.0,17.0,"(16.0, 23.0]",2.0,8.0,"(7.0, 11.0]",2.0,alexander place,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2441.0,2442.0,alexander place,long-term care home,Nursing and residential care facilities,Province of Ontario,NaN,329,parkside drive,L0R2H0,hamilton,on,NaN,Hamilton,3525005.0,35.0,43.342345,-79.900820
3,4,ALGOMA MANOR NURSING HOME,145 Dawson Street,"Thessalon, P0R1L0",North East,Algoma Manor Nursing Home,NaN,NaN,Home with approximately 96 beds,Yes,Yes,Yes,Yes,NaN,Thessalon,P0R1L0,96.0,23.0,"(21.0, 31.0]",1.0,14.0,"(11.0, 16.0]",1.0,7.0,"(5.0, 7.0]",1.0,algoma manor nursing home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2448.0,2449.0,algoma manor nursing home,long-term care home,Nursing and residential care facilities,Province of Ontario,NaN,145,dawson street,P0R1L0,thessalon,on,NaN,Thessalon,3557028.0,35.0,46.263462,-83.563249
4,5,ALGONQUIN NURSING HOME,207 Turcotte Park Road P.O. Box 270,"Mattawa, P0H1V0",North East,Algonquin Nursing Home Of Mattawa,Hôpital De Mattawa Hospital,Non-Profit,Home with approximately 73 beds,Yes,Yes,Yes,Yes,NaN,Mattawa,P0H1V0,73.0,23.0,"(21.0, 31.0]",1.0,14.0,"(11.0, 16.0]",1.0,5.0,"(0.0, 5.0]",0.0,algonquin nursing home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2455.0,2456.0,algonquin nursing home,long-term care home,Nursing and residential care facilities,Province of Ontario,NaN,231,10 street,P0H1V0,mattawa,on,NaN,Mattawa,3548021.0,35.0,46.313493,-78.719841


In [21]:
# Check that the number of beds from the general LTC database agree with number of beds from the COVID LTC database
df_all2 = df_all[df_all.Beds.notnull()]
# df_all2[['number of beds', 'Beds']]

# There are 19 homes where the number of beds in the 2 LTC databases do not agree
df_all2['bed_match'] = np.where((df_all2['number of beds'] == df_all2['Beds']), 'ok', 'no_match')
df_all2['bed_match'].value_counts()
# df_all2[['cleaned_name', 'number of beds', 'Beds']]





/Users/mds/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


ok          225
no_match    19 
Name: bed_match, dtype: int64

In [22]:
# review homes where value for number of beds differ between the databases
df_all3 = df_all2[df_all2['bed_match'] == 'no_match']
df_all3['diff_beds'] = df_all3['number of beds'] - df_all3['Beds']
df_all3[['cleaned_name', 'number of beds', 'Beds', 'diff_beds']]

/Users/mds/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,cleaned_name,number of beds,Beds,diff_beds
51,camilla care community,237.0,236.0,1.0
79,centre d'accueil roger seguin,110.0,113.0,-3.0
89,chartwell pine grove long term care residence,101.0,96.0,5.0
95,chartwell westmount long term care residence,161.0,160.0,1.0
96,chartwell white eagle long term care residence,57.0,56.0,1.0
147,elmwood place,78.0,90.0,-12.0
183,extendicare timmins,121.0,180.0,-59.0
226,glen hill strathaven,199.0,210.0,-11.0
231,golden years nursing home,90.0,88.0,2.0
279,ioof seniors home,188.0,182.0,6.0


## Manual review of some beds numbers discrepancies
- For kensington village, st. patrick's home, mount hope centre for long term care	
    - websites support the COVID LTC values
- Therefore where there is a discrepancy in the number of beds, will take the number from the covid LTC database

In [23]:
# create a new column with number of beds as defined above
df_all['number_beds'] = np.nan
df_all['number_beds'] = np.where((df_all.Beds.isnull()), df_all['number of beds'], df_all['Beds'])
df_all.iloc[[620, 499]]

,index_x,name,address,city and postal code,LHIN,licensee,management,home type,beds,short stay,residents council,family council,accreditation,additional_info,city_x,postal code,number of beds,total_inspections,quartiles_total_range,quartiles_total_rank,5y_inspections,quartiles_5y_range,quartiles_5y_rank,2y_inspections,quartiles_2y_range,quartiles_2y_rank,cleaned_name,Unnamed: 0,LTC Home,City,Beds,Confirmed Resident Cases,Resident Deaths,Confirmed Staff Cases,Status,level_0,index_y,facility_name,source_facility_type,odhf_facility_type,provider,unit,street_no,street_name,postal_code,city_y,province,source_format_str_address,CSDname,CSDuid,Pruid,latitude,longitude,number_beds
620,642,WOODLAND VILLA,30 Milles Roches Road R. R. #1,"Long Sault, K0C1P0",Champlain,0760444 B.C. Ltd. As General Partner On Behalf Of Omni Health Care Limited Partnership,NaN,For-Profit,Home with approximately 119 beds,No,Yes,Yes,Yes,NaN,Long Sault,K0C1P0,119.0,44.0,"(31.0, 46.0]",2.0,22.0,"(16.0, 23.0]",2.0,6.0,"(5.0, 7.0]",1.0,woodland villa,179.0,Woodland Villa,Long Sault,111.0,<5,0,0,Active,6097.0,6098.0,woodland villa,long-term care home,Nursing and residential care facilities,Province of Ontario,NaN,30,mille roches road,K0C1P0,long sault,on,NaN,South Stormont,3501011.0,35.0,45.033767,-74.887217,111.0
499,518,STRATHMERE LODGE,599 Albert Street Box 5000,"Strathroy, N7G3J3",South West,The Corporation Of The County Of Middlesex,NaN,Municipal,Home with approximately 160 beds,Yes,Yes,Yes,No,NaN,Strathroy,N7G3J3,160.0,42.0,"(31.0, 46.0]",2.0,19.0,"(16.0, 23.0]",2.0,7.0,"(5.0, 7.0]",1.0,strathmere lodge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5491.0,5492.0,strathmere lodge,long-term care home,Nursing and residential care facilities,Province of Ontario,NaN,599,albert street,N7G3J3,strathroy,on,NaN,Strathroy-Caradoc,3539015.0,35.0,42.955702,-81.643928,160.0


In [25]:
list_col = ['index_x', 'name', 'city and postal code', 'licensee', 'management',
            'beds', 'number of beds', 'additional_info', 'Unnamed: 0', 'LTC Home',
            'City','Beds','level_0','index_y','facility_name', 'source_facility_type', 
            'odhf_facility_type', 'provider', 'unit', 'street_no', 'street_name',
            'postal_code','city_y', 'province', 'source_format_str_address', 'Pruid'
            ]
df_trim = df_all.drop(list_col, axis = 1)
df_trim = df_trim.rename(columns = {'city_x':'city'})
df_trim.head()

,address,LHIN,home type,short stay,residents council,family council,accreditation,city,postal code,total_inspections,quartiles_total_range,quartiles_total_rank,5y_inspections,quartiles_5y_range,quartiles_5y_rank,2y_inspections,quartiles_2y_range,quartiles_2y_rank,cleaned_name,Confirmed Resident Cases,Resident Deaths,Confirmed Staff Cases,Status,CSDname,CSDuid,latitude,longitude,number_beds
0,1200 Afton Drive,Erie St. Clair,For-Profit,No,Yes,Yes,Yes,Sarnia,N7S6L6,67.0,"(46.0, 172.0]",3.0,34.0,"(23.0, 86.0]",3.0,9.0,"(7.0, 11.0]",2.0,afton park place long term care community,NaN,NaN,NaN,NaN,Sarnia,3538030.0,42.976325,-82.361444,128.0
1,5050 Hillside Drive,Hamilton Niagara Haldimand Brant (Hnhb),Non-Profit,No,Yes,Yes,No,Beamsville,L0R1B2,39.0,"(31.0, 46.0]",2.0,25.0,"(23.0, 86.0]",3.0,15.0,"(11.0, 44.0]",3.0,"albright gardens homes, incorporated",NaN,0,NaN,Inactive,NaN,NaN,NaN,NaN,231.0
2,329 Parkside Drive P. O. Box 50,Hamilton Niagara Haldimand Brant (Hnhb),For-Profit,Yes,Yes,Yes,Yes,Waterdown,L0R2H0,28.0,"(21.0, 31.0]",1.0,17.0,"(16.0, 23.0]",2.0,8.0,"(7.0, 11.0]",2.0,alexander place,NaN,NaN,NaN,NaN,Hamilton,3525005.0,43.342345,-79.900820,128.0
3,145 Dawson Street,North East,NaN,Yes,Yes,Yes,Yes,Thessalon,P0R1L0,23.0,"(21.0, 31.0]",1.0,14.0,"(11.0, 16.0]",1.0,7.0,"(5.0, 7.0]",1.0,algoma manor nursing home,NaN,NaN,NaN,NaN,Thessalon,3557028.0,46.263462,-83.563249,96.0
4,207 Turcotte Park Road P.O. Box 270,North East,Non-Profit,Yes,Yes,Yes,Yes,Mattawa,P0H1V0,23.0,"(21.0, 31.0]",1.0,14.0,"(11.0, 16.0]",1.0,5.0,"(0.0, 5.0]",0.0,algonquin nursing home,NaN,NaN,NaN,NaN,Mattawa,3548021.0,46.313493,-78.719841,73.0


In [28]:
df_trim.to_csv(r'df_LTCdata_for_stats_FINAL.csv', index = False)